In [1]:
from urllib.request import urlopen
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
import requests
from datetime import datetime
from openpyxl import load_workbook
from colorama import Fore, Back, Style #print colored text

In [2]:
start_date = datetime(2021,2,23)
end_date = datetime(2021,3,17)
page_range = range (1,3)
period = '?startTime=' + str(start_date.date()) + '&endTime=' + str(end_date.date())

dict_urls = { 
    '无锡300系不锈钢冷轧卷板': 'https://list.info.mybxg.com/market/p-665,666,667,2222,2257,2259------0-01010301-------1.html',
    '无锡300系不锈钢热轧卷板': 'https://list.info.mybxg.com/market/p-671-0----0102160102-0-01010301-------1.html',
    '300系不锈钢工业无缝管': 'https://list.info.mybxg.com/market/p-681-----0102160203-0--------1.html',
    '300系不锈钢工业焊管': 'https://list.info.mybxg.com/market/p-678-----010216020201-0--------1.html',
    '上海市场工角槽钢': 'https://list1.mysteel.com/market/p-227-0----01010701-0-010101-------1.html',
    '上海市场冷轧板卷': 'https://list1.mysteel.com/market/p-221-----01010401-0-010101-------1.html',
    '上海市场热轧板卷': 'https://list1.mysteel.com/market/p-221-----01010301-0-010101-------1.html',
    '上海市场低合金卷': 'https://list1.mysteel.com/market/p-221-----01010302-0-010101-------1.html',
    '上海市场镀锌板卷': 'https://list1.mysteel.com/market/p-435-----01010501-0-010101-------1.html',
    '上海市场无缝管': 'https://list1.mysteel.com/market/p-236-----01010901-0-010101-------1.html',
    '沈阳市场H型钢': 'https://list1.mysteel.com/market/p-223-----01010705-0-01050101-------1.html',
    '上海市场方矩管': 'https://list1.mysteel.com/market/p-221-----01010904-0-010101-------1.html',
    '长春市场工角槽钢': 'https://list1.mysteel.com/market/p-223-----01010701-0-01050201-------1.html',
    '长春市场冷轧板卷': 'https://list1.mysteel.com/market/p-223-----01010401-0-01050201-------1.html',
    '烟台市场热轧开平板': 'https://list1.mysteel.com/market/p-223-----01010306-0-01010205-------1.html',
    '不锈钢角钢': 'https://list.info.mybxg.com/market/p-691-----0102160303-0--------1.html'
    } 
dict_urls_1 = { 
    '无锡300系不锈钢冷轧卷板': 'https://list.info.mybxg.com/market/p-665,666,667,2222,2257,2259------0-01010301-------1.html',
    '无锡300系不锈钢热轧卷板': 'https://list.info.mybxg.com/market/p-671-0----0102160102-0-01010301-------1.html',
    '300系不锈钢工业无缝管': 'https://list.info.mybxg.com/market/p-681-----0102160203-0--------1.html',
    '300系不锈钢工业焊管': 'https://list.info.mybxg.com/market/p-678-----010216020201-0--------1.html',
    '不锈钢角钢': 'https://list.info.mybxg.com/market/p-691-----0102160303-0--------1.html'
    } 
dict_urls_2 = { 
    '上海市场工角槽钢': 'https://list1.mysteel.com/market/p-227-0----01010701-0-010101-------1.html',
    '上海市场冷轧板卷': 'https://list1.mysteel.com/market/p-221-----01010401-0-010101-------1.html',
    '上海市场热轧板卷': 'https://list1.mysteel.com/market/p-221-----01010301-0-010101-------1.html',
    '上海市场低合金卷': 'https://list1.mysteel.com/market/p-221-----01010302-0-010101-------1.html',
    '上海市场镀锌板卷': 'https://list1.mysteel.com/market/p-221-----01010501-0-010101-------1.html',
    '上海市场无缝管': 'https://list1.mysteel.com/market/p-221-----01010901-0-010101-------1.html',
    '沈阳市场H型钢': 'https://list1.mysteel.com/market/p-223-----01010705-0-01050101-------1.html',
    '上海市场方矩管': 'https://list1.mysteel.com/market/p-221-----01010904-0-010101-------1.html',
    '长春市场工角槽钢': 'https://list1.mysteel.com/market/p-223-----01010701-0-01050201-------1.html',
    '长春市场冷轧板卷': 'https://list1.mysteel.com/market/p-223-----01010401-0-01050201-------1.html',
    '烟台市场热轧开平板': 'https://list1.mysteel.com/market/p-223-----01010306-0-01010205-------1.html'
    } 
dict_urls_2 = {key: value + period for key, value in dict_urls_2.items()}

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.181 Safari/537.36',
    'Cookie': '_last_loginuname=viviyanwei; Hm_lvt_e14a1f00255ca6d026803cca20b08490=1607743514; href=https%3A%2F%2Finfo.mybxg.com%2Fm%2F20%2F1210%2F16%2F91D67724C337E319.html; accessId=5d36a9e0-919c-11e9-903c-ab24dbab411b; qimo_xstKeywords_5d36a9e0-919c-11e9-903c-ab24dbab411b=; _login_token=3191c6adf1eb4ce191bcbc278dc29301; _login_mid=2468603; _login_uid=1626926; _login_ip=116.228.72.50; 3191c6adf1eb4ce191bcbc278dc29301=11%3D5%261%3D10; Hm_lpvt_e14a1f00255ca6d026803cca20b08490=1607926822; qimo_seosource_5d36a9e0-919c-11e9-903c-ab24dbab411b=%E5%85%B6%E4%BB%96%E7%BD%91%E7%AB%99; qimo_seokeywords_5d36a9e0-919c-11e9-903c-ab24dbab411b=%E6%9C%AA%E7%9F%A5; pageViewNum=9',
}

In [3]:
for dict_url in dict_urls.keys():
    if dict_url in dict_urls_2.keys():
        parent_url = dict_urls_2.get(dict_url)
        req = requests.get(parent_url)
        mainPage = req.content
        mainSoup = BeautifulSoup(mainPage, "lxml")
        titles = []
        links = []
        frame_all = []
        all_href = mainSoup.findAll('a')

        for link in all_href:
            titles.append(link.get('title'))
        for link in all_href:
            links.append(link.get('href'))

        df_web = pd.DataFrame(zip(links, titles, links))
        df_web = df_web[df_web[1].str.contains(str(dict_url), na = False)]
        index = df_web.iloc[0][0].index('21')
        df_web[0] = df_web[0].str[index:index+2] + df_web[0].str[index+3:index+5] + df_web[0].str[index+5:index+7]
        df_web[0] = pd.to_datetime(df_web[0], format='%y%m%d').dt.date

        #Remove time in parenthesis
        df_web[1] = df_web[1].str.replace(r"\（.*\）","")
        df_web[1] = df_web[1].str.replace(r"\(.*\)","")
        #print(df_web)

        for j in range(len(df_web)):
            #print(df_web.iloc[j][0], df_web.iloc[j][1])
            if 'https' not in df_web.iloc[j][2]:
                url = 'https:' + df_web.iloc[j][2]
            else:
                url = df_web.iloc[j][2]

            response = requests.get(url, headers = headers)
            content = response.content
            soup = BeautifulSoup(content,'html5lib')
            text = soup.get_text()

            rows = soup.find_all('tr')
            for row in rows:
                row_td = row.find_all('td')

            str_cells = str(row_td)
            cleantext = BeautifulSoup(str_cells, "lxml").get_text()

            list_rows = []
            for row in rows:
                cells = row.find_all('td')
                str_cells = str(cells)
                clean = re.compile('<.*?>')
                clean2 = (re.sub(clean, '', str_cells))
                list_rows.append(clean2)

            df = pd.DataFrame(list_rows)
            df.drop(df.index[:15], inplace=True)
            df.drop([16], inplace=True)
            df = df.iloc[:-2]

            df1 = df[0].str.split(',', expand=True)

            for k in range(0,len(df1.columns)):
                df1[k] = df1[k].str.strip('[]\r\n\t请筛选 ')

            df1.insert(0, "Date", df_web.iloc[j][0]) 
            df1.at[15, 'Date'] = '日期'
            df1.columns = range(df1.shape[1])
            df1 = df1.reset_index(drop = True)
            df_title = df1.iloc[0:1]
            df1 = df1.iloc[1:]
            df1.columns = range(df1.shape[1])
            df1 = df1.reset_index(drop = True)
            #print(df1)
            frame_all.append(df1)
        df_frame = pd.concat(frame_all)
        #df_all = pd.concat([df_title, df_frame])
        df_frame.columns = range(df_frame.shape[1])
        df_frame = df_frame.reset_index(drop = True)
        print(Fore.GREEN, dict_url, Fore.WHITE, df_frame)

    elif dict_url in dict_urls_1.keys():
        frame_all = []
        for i in page_range:
            parent_url = dict_urls_1.get(dict_url)[:-6] + str(i) + dict_urls_1.get(dict_url)[-5:]
            req = requests.get(parent_url)
            mainPage = req.content
            mainSoup = BeautifulSoup(mainPage, "lxml")
            titles = []
            links = []
            all_href = mainSoup.findAll('a')

            for link in all_href:
                titles.append(link.get('title'))
            for link in all_href:
                links.append(link.get('href'))

            df_web = pd.DataFrame(zip(links, titles, links))
            df_web = df_web[df_web[1].str.contains(str(dict_url), na = False)]

            if not df_web.empty:
                index = df_web.iloc[0][0].index('.com/m/2')
                df_web[0] = df_web[0].str[index+7:index+9] + df_web[0].str[index+10:index+12] + df_web[0].str[index+12:index+14]
                df_web[0] = pd.to_datetime(df_web[0], format='%y%m%d')

                #Remove time in parenthesis
                df_web[1] = df_web[1].str.replace(r"\（.*\）","")
                df_web[1] = df_web[1].str.replace(r"\(.*\)","")

                for j in range (len(df_web)):
                    #print(df_web.iloc[j][0], df_web.iloc[j][1])
                    if 'https' not in df_web.iloc[j][2]:
                        url = 'https:' + df_web.iloc[j][2]
                    else:
                        url = df_web.iloc[j][2]

                    response = requests.get(url, headers = headers)

                    content = response.content
                    soup = BeautifulSoup(content,'html5lib')
                    text = soup.get_text()

                    rows = soup.find_all('tr')
                    for row in rows:
                        row_td = row.find_all('td')

                    str_cells = str(row_td)
                    cleantext = BeautifulSoup(str_cells, "lxml").get_text()

                    list_rows = []
                    for row in rows:
                        cells = row.find_all('td')
                        str_cells = str(cells)
                        clean = re.compile('<.*?>')
                        clean2 = (re.sub(clean, '', str_cells))
                        list_rows.append(clean2)

                    df = pd.DataFrame(list_rows)
                    df.drop([0,1], inplace=True)
                    df = df.iloc[:-7]

                    df1 = df[0].str.split(',', expand=True)

                    for k in range(0,len(df1.columns)):
                        df1[k] = df1[k].str.strip('[]\r\n\t请筛选 ')

                    df1.insert(0, "Date", df_web.iloc[j][0]) 
                    #df1.at[0, 'Date'] = '日期'
                    df1.columns = range(df1.shape[1])
                    df1 = df1.reset_index(drop = True)
                    df_title = df1.iloc[0:1]
                    #df1 = df1.iloc[1:]
                    df1.columns = range(df1.shape[1])
                    df1 = df1.reset_index(drop = True)
                    #print(df1)
                    frame_all.append(df1)
        df_frame = pd.concat(frame_all)
        #df_all = pd.concat([df_title, df_frame])
        df_frame = df_frame[df_frame[0].between(start_date,end_date)]
        df_frame.columns = range(df_frame.shape[1])
        df_frame = df_frame.reset_index(drop = True)
        print(Fore.YELLOW, dict_url, Fore.WHITE, df_frame)

    local_path = r'\\apac.duerr.int\SGH\DEPT\ADM-Purchasing\002_Commodities\000_Raw Material\2021 - Copy.xlsx'
    writer_to_local = pd.ExcelWriter(local_path, mode = 'w', engine = 'openpyxl')
    df_old = pd.read_excel(local_path, sheet_name = dict_url, header = None)
    book = load_workbook(local_path)
    writer_to_local.book = book
    writer_to_local.sheets = {ws.title: ws for ws in book.worksheets}
    df_frame.to_excel(writer_to_local, index = False, header = None, encoding = 'utf-8', sheet_name = dict_url, startrow = len(df_old))
    #df_all.to_excel(writer_to_local, index = False, header = None, encoding = 'utf-8', sheet_name = dict_url)
    writer_to_local.save()
    writer_to_local.close()

<ipython-input-3-8eb1733c21a8>:106: FutureWarning: The default value of regex will change from True to False in a future version.
  df_web[1] = df_web[1].str.replace(r"\（.*\）","")
<ipython-input-3-8eb1733c21a8>:107: FutureWarning: The default value of regex will change from True to False in a future version.
  df_web[1] = df_web[1].str.replace(r"\(.*\)","")


 无锡300系不锈钢冷轧卷板                0   1              2       3   4      5    6                7
0    2021-03-17  卷板     0.4*1219*C  304/2B  太钢  17300  -50  厚度≤0.37  加价1600
1    2021-03-17  卷板     0.5*1219*C  304/2B  太钢  17200  -50               切边
2    2021-03-17  卷板     0.6*1219*C  304/2B  太钢  17100  -50               切边
3    2021-03-17  卷板     0.7*1219*C  304/2B  太钢  17000  -50               切边
4    2021-03-17  卷板     0.8*1219*C  304/2B  太钢  16800  -50               切边
...         ...  ..            ...     ...  ..    ...  ...              ...
4534 2021-02-26  平板  1.2*1219*2438  321/2B  张浦  17550    -               切边
4535 2021-02-26  平板  1.5*1219*2438  321/2B  张浦  17200    -               切边
4536 2021-02-26  平板    2*1219*2438  321/2B  张浦  17050    -               切边
4537 2021-02-26  平板  2.5*1219*2438  321/2B  张浦  17450    -               切边
4538 2021-02-26  平板    3*1219*2438  321/2B  张浦  17700    -               切边

[4539 rows x 8 columns]


ValueError: File is not a recognized excel file